In [1]:
import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
import numpy as np
import pandas as pd
import pandas_datareader.data as web
from datetime import datetime as dt

# df = web.DataReader('005930', 'naver', start='2015-01-01', end=dt.now())
# df.tail()

In [22]:
# df = pd.read_csv('kospi_data.csv')
# col = ['기업명', '종목코드']
# df1 = df[col].copy()
# df1.columns = ['label','value']
# df1.head()
# stockid = df1.to_dict('records')
# stockid

df = pd.read_csv('kospi_data.csv')
col = ['기업명', '종목코드']
df1 = df[col].copy()
df1.columns = ['label','value']
df1['value'] = df1['value'].apply(str)
df1['value'] = df1['value'].str.zfill(6)
stock_data = df1.to_dict('records')
stock_data

list1=[
        {'label': 'Samsung', 'value': '005930'},
        {'label': 'SKInnovation', 'value': '096770'}
        ]
print(list1)
print(stock_data[:2])

[{'label': 'Samsung', 'value': '005930'}, {'label': 'SKInnovation', 'value': '096770'}]
[{'label': 'AJ네트웍스', 'value': '095570'}, {'label': 'AK홀딩스', 'value': '006840'}]


In [41]:
# import requests
# from bs4 import BeautifulSoup


# def get_balance_sheet(stockid):
#     URL = "https://finance.naver.com/item/main.nhn?code="+str(stockid)

#     samsung_electronic = requests.get(URL)
#     html = samsung_electronic.text

#     soup = BeautifulSoup(html, 'html.parser')

#     finance_html = soup.select('div.section.cop_analysis div.sub_section')[0]

#     th_data = [item.get_text().strip() for item in finance_html.select('thead th')]
#     annual_date = th_data[3:7]
#     quarter_date = th_data[7:13]

#     finance_index = [item.get_text().strip() for item in finance_html.select               ('th.h_th2')][3:]

#     finance_data = [item.get_text().strip() for item in finance_html.select('td')]

#     import numpy as np

#     finance_data = np.array(finance_data)
#     finance_data.resize(len(finance_index), 10)

#     finance_date = annual_date + quarter_date

#     import pandas as pd
#     finance = pd.DataFrame(data=finance_data[0:,0:], index=finance_index,                  columns=finance_date)

#     annual_finance = finance.iloc[:, :4]
#     quarter_finance = finance.iloc[:, 4:]

In [23]:
import pandas as pd
import requests

# Instead of using beautifulsoup, the process can be done in a much simpler way as below using pd.read_html()

def get_balance_sheet(stockid):
    URL = "https://finance.naver.com/item/main.nhn?code="+str(stockid)

    samsung_electronic = requests.get(URL)
    html = samsung_electronic.text

    financial_stmt = pd.read_html(samsung_electronic.text)[3]

    # financial_stmt.set_index(('주요재무정보', '주요재무정보', '주요재무정보'), inplace=True)
    # financial_stmt.set_index(('주요재무정보'), inplace=True)

    financial_stmt.index.rename('주요재무정보', inplace=True)
    financial_stmt.columns = financial_stmt.columns.droplevel(2)
    financial_stmt.columns = financial_stmt.columns.droplevel(0)

    df = pd.DataFrame(financial_stmt)
    # balance_sheet.reset_index(level=0,inplace=True)
    # cols = list(df.columns.values)
    cols = ['주요재무정보','2017.12','2018.12','2019.03','2019.06','2019.09','2019.12','2020.03','2020.06(E)','2020.12(E)']
    df = df[cols]
    df.drop(['2017.12','2018.12','2020.12(E)'],axis = 1,inplace=True)
    df.rename(columns={'주요재무정보':'Indicators'},inplace=True)
    df = df.set_index('Indicators')
    df.columns = ['2019.03','2019.06','2019.09','2019.12(A)','2019.12','2020.03','2020.06(E)']
    df = df[['2019.12(A)','2019.03','2019.06','2019.09','2019.12','2020.03','2020.06(E)']]
    df.drop(df.index[[13,14,15]],inplace=True)
    df.drop('2019.12(A)',axis = 1,inplace=True)
    col = df.columns.values
    df[col] = df[col].astype(float)
    df = df.round(2)
    df.reset_index(inplace=True)
    df['Indicators'] = df['Indicators'].replace({'매출액': 'Sales(100mil Won)', '영업이익': 'Operating Revenue(100mil Won)', '당기순이익': 'Net Income(100mil Won)', '영업이익률': 'Operating Revenue(%)','순이익률': 'Net Income(%)', 'ROE(지배주주)': 'ROE', '부채비율': 'Liabilities(%)', '당좌비율': 'Quick Assets(%)', '유보율': 'Reserve Ratio(%)', 'EPS(원)': 'EPS(Won)', 'PER(배)': 'PER', 'BPS(원)': 'BPS(Won)', 'PBR(배)': 'PBR'})
    return df

df = get_balance_sheet('005930')
df

,Indicators,2019.03,2019.06,2019.09,2019.12,2020.03,2020.06(E)
0,Sales(100mil Won),523855.00,561271.00,620035.00,598848.00,553252.00,511401.00
1,Operating Revenue(100mil Won),62333.00,65971.00,77779.00,71603.00,64473.00,64703.00
2,Net Income(100mil Won),50436.00,51806.00,62877.00,52270.00,48849.00,52746.00
3,Operating Revenue(%),11.90,11.75,12.54,11.96,11.65,12.65
4,Net Income(%),9.63,9.23,10.14,8.73,8.83,10.31
5,ROE,16.21,13.23,10.05,8.69,8.45,NaN
6,Liabilities(%),36.27,33.05,34.14,34.12,34.19,NaN
7,Quick Assets(%),209.25,230.74,235.80,233.57,237.80,NaN
8,Reserve Ratio(%),27832.98,28129.35,28541.64,28856.02,29134.12,NaN
9,EPS(Won),752.00,746.00,899.00,770.00,720.00,735.00


In [27]:
df = pd.read_csv('kospi_data.csv')
col = ['기업명', '종목코드']
df1 = df[col].copy()
df1.columns = ['label','value']
df1['value'] = df1['value'].apply(str)
df1['value'] = df1['value'].str.zfill(6)
df1['label'] = df1['label']+' ('+df1['value']+')'
stock_data = df1.to_dict('records')
# stock_data
df1

,label,value
0,AJ네트웍스095570,095570
1,AK홀딩스006840,006840
2,BGF027410,027410
3,BGF리테일282330,282330
4,BNK금융지주138930,138930
5,BYC001460,001460
6,CJ001040,001040
7,CJ CGV079160,079160
8,CJ대한통운000120,000120
9,CJ씨푸드011150,011150
